# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [10]:
print("Your API Key:", params["apiKey"])

Your API Key: Your_GEOAPIFY_API_KEY


In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port elizabeth,-33.9180,25.5701,24.16,92,100,8.23,ZA,2025-01-24 07:52:47
1,1,bonthe,7.5264,-12.5050,26.10,74,95,1.95,SL,2025-01-24 07:52:47
2,2,qaqortoq,60.7167,-46.0333,-15.90,82,100,2.71,GL,2025-01-24 07:52:47
3,3,anadyr,64.7500,177.4833,-23.72,76,100,13.86,RU,2025-01-24 07:52:47
4,4,waitangi,-43.9535,-176.5597,14.90,94,34,1.79,NZ,2025-01-24 07:50:16


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    x="Lon",
    y="Lat",
    size="Humidity",
    color="City",
    cmap="Category20",
    frame_width=800,
    frame_height=500,
    alpha=0.7,
    tiles="OSM",
    hover_cols=["City"]
).opts(
    xlabel="Longitude",
    ylabel="Latitude",
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [x,y]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df[
    (city_data_df["Humidity"] >= 30) & (city_data_df["Humidity"] <= 50) &
    (city_data_df["Wind Speed"] >= 1) & (city_data_df["Wind Speed"] <=5) &
    (city_data_df["Cloudiness"] >= 0) & (city_data_df["Cloudiness"] <=50)
]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df.head(15)

,City_ID,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
27,27,blackmans bay,-43.0167,147.3167,20.41,50,6,1.34,AU,2025-01-24 07:52:53
36,36,new norfolk,-42.7826,147.0587,21.07,47,2,3.62,AU,2025-01-24 07:52:55
44,44,bredasdorp,-34.5322,20.0403,27.67,42,0,2.78,ZA,2025-01-24 07:52:57
72,72,petropavlovsk-kamchatsky,53.0452,158.6483,-8.67,38,0,2.00,RU,2025-01-24 07:53:03
78,78,price,39.5994,-110.8107,-8.41,48,0,4.12,US,2025-01-24 07:53:05
81,81,ushuaia,-54.8000,-68.3000,10.81,46,20,3.60,AR,2025-01-24 07:51:13
103,103,elamanchili,17.5500,82.8667,29.65,38,0,3.62,IN,2025-01-24 07:53:10
113,113,ifrane,33.3333,-5.2500,5.22,45,0,2.45,MA,2025-01-24 07:53:12
120,120,hermanus,-34.4187,19.2345,24.89,49,0,2.70,ZA,2025-01-24 07:53:14
123,123,morelos,18.7500,-99.0000,17.08,45,1,1.45,MX,2025-01-24 07:53:14


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[["City", "Country", "Lat", "Lon", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(15)

,City,Country,Lat,Lon,Humidity,Hotel Name
27,blackmans bay,AU,-43.0167,147.3167,50,
36,new norfolk,AU,-42.7826,147.0587,47,
44,bredasdorp,ZA,-34.5322,20.0403,42,
72,petropavlovsk-kamchatsky,RU,53.0452,158.6483,38,
78,price,US,39.5994,-110.8107,48,
81,ushuaia,AR,-54.8000,-68.3000,46,
103,elamanchili,IN,17.5500,82.8667,38,
113,ifrane,MA,33.3333,-5.2500,45,
120,hermanus,ZA,-34.4187,19.2345,49,
123,morelos,MX,18.7500,-99.0000,45,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
import requests

# Set parameters to search for a hotel
radius = 10000  # Search within 10 km
base_url = "https://api.geoapify.com/v2/places"

params = {
    "categories": "accommodation.hotel",  # Look for hotels
    "limit": 1,  # Get only the nearest hotel
    "apiKey": "Your_GEOAPIFY_API_KEY"  # Replace with your actual API key
}

# Print a message to follow up on the hotel search
print("Starting hotel search...")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    lat, lon = row["Lat"], row["Lon"]

    # Create a copy of params to avoid modifying the global dictionary
    query_params = params.copy()
    query_params["filter"] = f"circle:{lon},{lat},{radius}"
    query_params["bias"] = f"proximity:{lon},{lat}"

    # Make an API request using the query_params dictionary
    response = requests.get(base_url, params=query_params)

    # Check if the request was successful
    if response.status_code == 200:
        # Convert the API response to JSON format
        name_address = response.json()

        # Grab the first hotel from the results and store the name in the hotel_df DataFrame
        try:
            hotel_name = name_address["features"][0]["properties"]["name"]
            hotel_df.loc[index, "Hotel Name"] = hotel_name  # Assign correctly
        except (KeyError, IndexError):
            # If no hotel is found, set the hotel name as "No hotel found".
            hotel_df.loc[index, "Hotel Name"] = "No hotel found"
    else:
        print(f"API request failed for {row['City']} (Status Code: {response.status_code})")
        hotel_df.loc[index, "Hotel Name"] = "API request failed"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - Nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()


Starting hotel search...
API request failed for blackmans bay (Status Code: 401)
blackmans bay - Nearest hotel: API request failed
API request failed for new norfolk (Status Code: 401)
new norfolk - Nearest hotel: API request failed
API request failed for bredasdorp (Status Code: 401)
bredasdorp - Nearest hotel: API request failed
API request failed for petropavlovsk-kamchatsky (Status Code: 401)
petropavlovsk-kamchatsky - Nearest hotel: API request failed
API request failed for price (Status Code: 401)
price - Nearest hotel: API request failed
API request failed for ushuaia (Status Code: 401)
ushuaia - Nearest hotel: API request failed
API request failed for elamanchili (Status Code: 401)
elamanchili - Nearest hotel: API request failed
API request failed for ifrane (Status Code: 401)
ifrane - Nearest hotel: API request failed
API request failed for hermanus (Status Code: 401)
hermanus - Nearest hotel: API request failed
API request failed for morelos (Status Code: 401)
morelos - Neare

,City,Country,Lat,Lon,Humidity,Hotel Name
27,blackmans bay,AU,-43.0167,147.3167,50,API request failed
36,new norfolk,AU,-42.7826,147.0587,47,API request failed
44,bredasdorp,ZA,-34.5322,20.0403,42,API request failed
72,petropavlovsk-kamchatsky,RU,53.0452,158.6483,38,API request failed
78,price,US,39.5994,-110.8107,48,API request failed


In [8]:
print("Your API Key:", params["apiKey"])

Your API Key: Your_GEOAPIFY_API_KEY


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE